topic modeling: statistical modeling for discovering the abstract "topics" that occur in a collection of documents

LDA(Latent Dirichlet Allocation): an example of topic model, used to classify text to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions.

In this project, I will practice applying LDA to a set of documents and split them into topics.

**Agenda**:
* the data
* data pre-processing
* bag of words on the data set
* TF-IDF
* running LDA using bag-of-words
* running LDA using TF-IDF
* performance evaluation by classifying sample document using LDA bag of words model
* performance evaluation by classifying sample document using LDA TF-IDF model
* testing model on unseen document

**The Data**

The dataset contains over one millioin news headlines published over 15 years.

In [5]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [11]:
data_text['headline_text'][0]

'aba decides against community broadcasting licence'

In [7]:
len(documents)

1103663

**Data Pre-processing**

steps:
* tokenization: split text into sentences and sentences into words; lowercase words; remove punctuation
* remove words with fewer than 3 char
* remove stopwords
* lemmatize words: words in third person are changed to first person; verbs in past and future tenses are changed into present
* stem words: words are reduced to their root form

In [19]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/ruihu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

perform lemmatize and stem preprocessing steps on text

In [21]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

remove stopwords and set threshold for char length

In [16]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

select a document to preview after preprocessing

In [22]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


preprocess the headlines, save results as 'processed_docs'

In [24]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

**Bag of words on the datasets**

create a dictionary containing the number of times a word appears in the training set

In [28]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


'overtim'

filter out tokens that appear in:
* less than 15 documents
* or more than 50% of the total corpus size
* keep the first 100000 most frequent tokens

In [26]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

gensim doc2bow: for each document we create a dictionary reporting how many words and how many times those words appear, save this to 'bow_corpus'

In [62]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]
#word id as key, word freq as value

[(77, 1), (116, 1), (495, 1), (4452, 1)]

preview bag of words for sample processed_docs

In [61]:
bow_doc_4310 = bow_corpus[8]

for i in range(len(bow_doc_4310)):
    print("word {} (\"{}\") appears in {} time".format(bow_doc_4310[i][0],
                                                      dictionary[bow_doc_4310[i][0]],
                                                      bow_doc_4310[i][1]))

word 11 ("aust") appears in 1 time
word 33 ("wast") appears in 1 time
word 34 ("address") appears in 1 time
word 35 ("council") appears in 1 time
word 36 ("iraq") appears in 1 time


**TF-IDF**

tf-idf: term frequency-inverse document frequency

create tf-idf model object using models

In [63]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

preview TF-IDF scores for sample document

In [64]:
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903603121911333),
 (1, 0.3852450692300274),
 (2, 0.4974556050119205),
 (3, 0.505567858418396)]


**Running LDA using Bag of Words** 

reference: http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/

In [65]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

id2word: mapping from word ids to words. it is used to determine the vocabulary size, as well as for debugging and topic printing

passes: number of passes through the corpus during training

for each topic, we will explore the words occuring in that topic and its relative weight.

In [66]:
for idx, topic in lda_model.print_topics(-1):
    print('topic: {} \nwords: {}'.format(idx, topic))

topic: 0 
words: 0.024*"south" + 0.019*"australia" + 0.019*"test" + 0.016*"tasmania" + 0.014*"chang" + 0.014*"turnbul" + 0.013*"break" + 0.010*"john" + 0.009*"say" + 0.009*"storm"
topic: 1 
words: 0.014*"brisban" + 0.014*"council" + 0.013*"final" + 0.012*"need" + 0.010*"review" + 0.009*"say" + 0.009*"premier" + 0.008*"law" + 0.008*"defend" + 0.008*"parliament"
topic: 2 
words: 0.031*"trump" + 0.018*"world" + 0.018*"open" + 0.017*"market" + 0.016*"nation" + 0.015*"australian" + 0.013*"rise" + 0.012*"record" + 0.012*"price" + 0.012*"share"
topic: 3 
words: 0.032*"govern" + 0.020*"school" + 0.020*"coast" + 0.014*"high" + 0.013*"north" + 0.013*"gold" + 0.013*"children" + 0.012*"student" + 0.012*"protest" + 0.010*"west"
topic: 4 
words: 0.030*"sydney" + 0.023*"polic" + 0.022*"adelaid" + 0.016*"interview" + 0.015*"shoot" + 0.014*"home" + 0.013*"woman" + 0.013*"melbourn" + 0.013*"die" + 0.013*"death"
topic: 5 
words: 0.021*"queensland" + 0.013*"live" + 0.013*"fund" + 0.013*"health" + 0.012*"r

**Running LDA using TF-IDF**

In [71]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [68]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('topic: {} \nword: {}'.format(idx, topic))

topic: 0 
word: 0.020*"trump" + 0.008*"grandstand" + 0.006*"abbott" + 0.005*"toni" + 0.005*"islam" + 0.005*"say" + 0.005*"raid" + 0.005*"syria" + 0.005*"suicid" + 0.005*"bomb"
topic: 1 
word: 0.012*"polic" + 0.009*"woman" + 0.009*"charg" + 0.008*"murder" + 0.008*"death" + 0.007*"search" + 0.007*"miss" + 0.006*"stab" + 0.006*"court" + 0.005*"septemb"
topic: 2 
word: 0.014*"queensland" + 0.007*"dollar" + 0.006*"monday" + 0.006*"august" + 0.006*"thursday" + 0.006*"sexual" + 0.005*"malcolm" + 0.005*"quiz" + 0.004*"western" + 0.004*"onlin"
topic: 3 
word: 0.007*"weather" + 0.005*"result" + 0.005*"leagu" + 0.005*"wild" + 0.004*"david" + 0.004*"blog" + 0.004*"live" + 0.004*"origin" + 0.004*"rugbi" + 0.004*"breakfast"
topic: 4 
word: 0.020*"countri" + 0.018*"hour" + 0.011*"market" + 0.008*"donald" + 0.008*"share" + 0.007*"price" + 0.006*"energi" + 0.005*"australian" + 0.005*"dairi" + 0.005*"cattl"
topic: 5 
word: 0.015*"interview" + 0.013*"crash" + 0.007*"kill" + 0.006*"asylum" + 0.005*"fatal"

**Performance evaluation by classifying sample document using LDA bag of words model**

check where our sample document would be classified

In [69]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [70]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nscore: {}\t \ntopic: {}".format(score, lda_model.print_topic(index, 10)))


score: 0.45864659547805786	 
topic: 0.024*"south" + 0.019*"australia" + 0.019*"test" + 0.016*"tasmania" + 0.014*"chang" + 0.014*"turnbul" + 0.013*"break" + 0.010*"john" + 0.009*"say" + 0.009*"storm"

score: 0.381317675113678	 
topic: 0.017*"crash" + 0.016*"miss" + 0.013*"peopl" + 0.013*"island" + 0.011*"driver" + 0.011*"polic" + 0.010*"forc" + 0.010*"want" + 0.010*"royal" + 0.010*"search"

score: 0.020012065768241882	 
topic: 0.031*"trump" + 0.018*"world" + 0.018*"open" + 0.017*"market" + 0.016*"nation" + 0.015*"australian" + 0.013*"rise" + 0.012*"record" + 0.012*"price" + 0.012*"share"

score: 0.0200088769197464	 
topic: 0.023*"kill" + 0.019*"countri" + 0.017*"china" + 0.016*"hour" + 0.012*"road" + 0.010*"week" + 0.010*"attack" + 0.010*"korea" + 0.010*"drum" + 0.009*"north"

score: 0.02000686153769493	 
topic: 0.021*"queensland" + 0.013*"live" + 0.013*"fund" + 0.013*"health" + 0.012*"rural" + 0.010*"servic" + 0.009*"plan" + 0.009*"worker" + 0.009*"busi" + 0.008*"industri"

score: 0.0

our test document has the highest probability to be topic 0.

**Performance evaluation by classifying sample document using LDA TF-IDF model**

In [72]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nscore: {}\t \ntopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


score: 0.5392301678657532	 
topic: 0.011*"crash" + 0.009*"queensland" + 0.009*"podcast" + 0.008*"turnbul" + 0.008*"donald" + 0.007*"flood" + 0.007*"road" + 0.007*"kill" + 0.006*"fatal" + 0.005*"truck"

score: 0.300755113363266	 
topic: 0.008*"sport" + 0.007*"hobart" + 0.006*"marriag" + 0.006*"decemb" + 0.005*"father" + 0.004*"bash" + 0.004*"condit" + 0.004*"polic" + 0.004*"islam" + 0.004*"rememb"

score: 0.020003454759716988	 
topic: 0.010*"govern" + 0.010*"rural" + 0.008*"market" + 0.006*"health" + 0.006*"fund" + 0.006*"nation" + 0.006*"news" + 0.005*"budget" + 0.005*"council" + 0.005*"share"

score: 0.020002439618110657	 
topic: 0.023*"countri" + 0.021*"hour" + 0.006*"friday" + 0.006*"ash" + 0.006*"domest" + 0.005*"violenc" + 0.005*"coal" + 0.003*"biki" + 0.003*"grand" + 0.003*"kimberley"

score: 0.020002136006951332	 
topic: 0.009*"live" + 0.006*"cattl" + 0.006*"malcolm" + 0.005*"august" + 0.004*"updat" + 0.004*"know" + 0.004*"export" + 0.004*"spring" + 0.004*"care" + 0.004*"donat"

our test document has the highest probability to be topic 0

**Testing model on unseen document**

In [75]:
unseen_doc = 'hello world'
bow_vector = dictionary.doc2bow(preprocess(unseen_doc))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("score: {}\t topic: {}".format(score, lda_model.print_topic(index, 5)))

score: 0.6999759674072266	 topic: 0.024*"south" + 0.019*"australia" + 0.019*"test" + 0.016*"tasmania" + 0.014*"chang"
score: 0.033357299864292145	 topic: 0.031*"trump" + 0.018*"world" + 0.018*"open" + 0.017*"market" + 0.016*"nation"
score: 0.03333336487412453	 topic: 0.026*"elect" + 0.018*"say" + 0.014*"donald" + 0.013*"labor" + 0.010*"bank"
score: 0.03333333507180214	 topic: 0.014*"brisban" + 0.014*"council" + 0.013*"final" + 0.012*"need" + 0.010*"review"
score: 0.03333333507180214	 topic: 0.032*"govern" + 0.020*"school" + 0.020*"coast" + 0.014*"high" + 0.013*"north"
score: 0.03333333507180214	 topic: 0.030*"sydney" + 0.023*"polic" + 0.022*"adelaid" + 0.016*"interview" + 0.015*"shoot"
score: 0.03333333507180214	 topic: 0.021*"queensland" + 0.013*"live" + 0.013*"fund" + 0.013*"health" + 0.012*"rural"
score: 0.03333333507180214	 topic: 0.023*"kill" + 0.019*"countri" + 0.017*"china" + 0.016*"hour" + 0.012*"road"
score: 0.03333333507180214	 topic: 0.017*"crash" + 0.016*"miss" + 0.013*"peo